In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint
from urllib.request import urlopen
import plotly.express as px

In [1]:
covid_file = 'covid_merged.csv'
politics_file = 'sars_political_leaning_nghia.csv'
fips_file = 'Resources/county_fips.csv'
deaths_file = 'Resources/sars_deaths.csv'

In [ ]:
covid_df = pd.read_csv(covid_file)
county_politics = pd.read_csv(politics_file)
county_fips = pd.read_csv(fips_file)
total_deaths = pd.read_csv(deaths_file)
#county_politics

In [ ]:
cases_df = covid_df[['County', 'Age 0-19_cases', 'Age 20-39_cases', 'Age 40-59_cases', 'Age 60-79_cases', 'Age 80+_cases']].groupby('County').sum()
hospital_df = covid_df[['County', 'Age 0-19_hospitalization', 'Age 20-39_hospitalization', 'Age 40-59_hospitalization', 'Age 60-79_hospitalization', 'Age 80+_hospitalization']].groupby('County').sum()
death_df = covid_df[['County', 'Age 0-19_death', 'Age 20-39_death', 'Age 40-59_death', 'Age 60-79_death', 'Age 80+_death']].groupby('County').sum()

In [ ]:
cases_df['Total Cases'] = cases_df.sum(axis=1)
hospital_df['Total Hospitalizations'] = hospital_df.sum(axis=1)
death_df['Total Deaths'] = death_df.sum(axis=1)

In [ ]:
county_df = pd.merge(county_politics, county_fips, on='County', how='left')
county_df

county_df = pd.merge(county_df, cases_df, on='County')
county_df = pd.merge(county_df, hospital_df, on='County')
county_df = pd.merge(county_df, death_df, on='County')

county_df = county_df[['Counter', 'County', 'Political_Party', 'fips', 'Total Cases', 'Total Hospitalizations', 'Total Deaths']]
#county_df

In [ ]:
wash_lat = 47.7511
wash_lon = -120.7401

with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [ ]:
fig = px.choropleth_mapbox(
    county_df,
    geojson=counties, 
    locations='fips', 
    color='Total Deaths',
    color_continuous_scale=['yellow', 'red'],
    mapbox_style='carto-positron',
    zoom=5.5,
    center= {"lat": wash_lat, "lon": wash_lon},
    opacity=0.5,
    labels={'County': 'Total Deaths'}
)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig

In [ ]:
fig = px.choropleth_mapbox(
    county_df, 
    geojson=counties, 
    locations='fips', 
    color='Political_Party',
    hover_name='County',
    color_discrete_map={
        'Republican': 'red',
        'Democratic': 'blue'
    },
    #color_continuous_scale='Viridis',
    mapbox_style='carto-positron',
    zoom=5.5,
    center= {"lat": 47.7511, "lon": -120.7401},
    opacity=0.5,
    labels={'Political_Party':'Political_Party'},
    title='Political Affiliation by County'
)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
#fig